In [1]:
import requests
import json
import pandas as pd
import numpy as np
import pymysql

TMDB API에서 영화id를 이용하여 각 영화의 keywords를 가져올 수 있음. 

https://api.themoviedb.org/3/movie/{movie_id}/keywords?api_key=36927ad1d2817ff0fa31947e47d186fd

In [2]:
#영화id가 512195인 영화의 키워드 크롤링 => 딕셔너리에 영화id와 keywords를 담고 있고 각 keyword는 고유id와 이름값을 딕셔너리에 담겨 리스트로 나열되어 있음.
keywords_url="https://api.themoviedb.org/3/movie/512195/keywords?api_key=36927ad1d2817ff0fa31947e47d186fd"
response = requests.get(keywords_url)
json_object = response.content
keyword_data=json.loads(json_object)
keyword_data

{'id': 512195,
 'keywords': [{'id': 1812, 'name': 'fbi'},
  {'id': 6710, 'name': 'interpol'},
  {'id': 10051, 'name': 'heist'},
  {'id': 14789, 'name': 'art thief'}]}

In [3]:
keyword_list = keyword_data["keywords"]
keyword_list

[{'id': 1812, 'name': 'fbi'},
 {'id': 6710, 'name': 'interpol'},
 {'id': 10051, 'name': 'heist'},
 {'id': 14789, 'name': 'art thief'}]

In [4]:
# 영화id를 사용하기 위해 영화정보 파일가져오기 
tmdb_movie_list = pd.read_csv('../data-files/total_tmdbmovielist.csv',index_col=0) 

C:\Users\user\.conda\envs\ykseo\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1,4,7,12,17,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
tmdb_movie_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 630159 entries, 0 to 630185
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   adult                  630158 non-null  object 
 1   backdrop_path          150102 non-null  object 
 2   belongs_to_collection  11995 non-null   object 
 3   budget                 630157 non-null  object 
 4   genres                 630155 non-null  object 
 5   homepage               69801 non-null   object 
 6   id                     630156 non-null  object 
 7   imdb_id                396414 non-null  object 
 8   original_language      630157 non-null  object 
 9   original_title         630130 non-null  object 
 10  overview               25238 non-null   object 
 11  popularity             630119 non-null  object 
 12  poster_path            426255 non-null  object 
 13  production_companies   630119 non-null  object 
 14  production_countries   630088 non-nu

In [6]:
a=tmdb_movie_list[['id','homepage','overview','title']].dropna()
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7351 entries, 2 to 629361
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7351 non-null   object
 1   homepage  7351 non-null   object
 2   overview  7351 non-null   object
 3   title     7351 non-null   object
dtypes: object(4)
memory usage: 287.1+ KB


In [16]:

b = a[["id"]] #영화id

all_keywords_list = []
error_list = []
for idx,  movie_id in enumerate(b["id"].values):  # movie_id 목록을 기준으로 인덱스와 movie_id를 반환
    try :
        keywords_url="https://api.themoviedb.org/3/movie/{0}/keywords?api_key=36927ad1d2817ff0fa31947e47d186fd"
        response = requests.get( keywords_url.format(movie_id))
        json_object = response.content
        keywords_data = json.loads(json_object)
        keywords_list = keywords_data["keywords"]

        for keyword in keywords_list:    
            keyword["movie_id"] = movie_id                                  
            all_keywords_list.append(keyword)

    except:
        error_list.append(movie_id)

    #if idx == 1000:
      #break

In [17]:
keywords=pd.DataFrame(all_keywords_list)
keywords

,id,name,movie_id
0,612,hotel,5.0
1,613,new year's eve,5.0
2,616,witch,5.0
3,622,bet,5.0
4,922,hotel room,5.0
...,...,...,...
37997,282965,genoa,876716.0
37998,287663,portorosso,876716.0
37999,886,movie business,877657.0
38000,33490,filmmaking,877657.0


In [18]:
keyword_christmas=keywords[keywords['name'] == "christmas"]
keyword_christmas.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 1859 to 37936
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        101 non-null    int64 
 1   name      101 non-null    object
 2   movie_id  101 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.2+ KB


In [19]:
keyword_christmas


,id,name,movie_id
1859,207317,christmas,508.0
2102,207317,christmas,562.0
2723,207317,christmas,671.0
3506,207317,christmas,850.0
4384,207317,christmas,1374.0
...,...,...,...
37257,207317,christmas,9479.0
37501,207317,christmas,9714.0
37788,207317,christmas,9969.0
37866,207317,christmas,810873.0


In [22]:
keyword_holiday=keywords[keywords['name'] == "holiday"]
keyword_holiday

,id,name,movie_id
3497,65,holiday,850.0
4709,65,holiday,1581.0
7618,65,holiday,10147.0
8812,65,holiday,11798.0
9519,65,holiday,13187.0
9670,65,holiday,13398.0
12594,65,holiday,17979.0
19466,65,holiday,338964.0
20084,65,holiday,346650.0
20401,65,holiday,353069.0


In [28]:
keyword_christmas_holiday=pd.merge(keyword_holiday, keyword_christmas, on="movie_id", how="inner") 
keyword_christmas_holiday

,id_x,name_x,movie_id,id_y,name_y
0,65,holiday,850.0,207317,christmas
1,65,holiday,1581.0,207317,christmas
2,65,holiday,10147.0,207317,christmas
3,65,holiday,13187.0,207317,christmas
4,65,holiday,13398.0,207317,christmas
5,65,holiday,17979.0,207317,christmas
6,65,holiday,338964.0,207317,christmas
7,65,holiday,367544.0,207317,christmas
8,65,holiday,410314.0,207317,christmas
9,65,holiday,419680.0,207317,christmas
